# 实验四

崔士强 PB22151743

## 任务一

In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv('data2.csv')

### Q1

In [8]:
columns_with_missing_values = df.columns[df.isnull().any()].tolist()

# Print the column names with missing values
print("Columns with missing values:")
print(columns_with_missing_values)

# Remove the rows with missing values
df = df.dropna()

Columns with missing values:
['node-caps', 'breast-quad']


### Q2

In [9]:
print("Value distribution for 'tumor-size' before corrections:")
print(df['tumor-size'].value_counts())

print("\nValue distribution for 'inv-nodes' before corrections:")
print(df['inv-nodes'].value_counts())

# Define a correction mapping for date-like entries
correction_map_tumor_size = {
    '14-Oct' : '10-14',
    '9-May' : '5-9'
}

correction_map_inv_nodes = {
    '5-Mar' : '3-5',
    '8-Jun' : '6-8',
    '11-Sep' : '9-11',
    '14-Dec' : '12-14'
}

# Apply corrections
df['tumor-size'] = df['tumor-size'].replace(correction_map_tumor_size)
df['inv-nodes'] = df['inv-nodes'].replace(correction_map_inv_nodes)

# Verify the corrections
print("\nCorrected value distribution for 'tumor-size':")
print(df['tumor-size'].value_counts())

print("\nCorrected value distribution for 'inv-nodes':")
print(df['inv-nodes'].value_counts())


Value distribution for 'tumor-size' before corrections:
tumor-size
30-34     57
25-29     51
20-24     48
15-19     29
14-Oct    28
40-44     22
35-39     19
0-4        8
50-54      8
9-May      4
45-49      3
Name: count, dtype: int64

Value distribution for 'inv-nodes' before corrections:
inv-nodes
0-2       209
5-Mar      34
8-Jun      17
11-Sep      7
15-17       6
14-Dec      3
24-26       1
Name: count, dtype: int64

Corrected value distribution for 'tumor-size':
tumor-size
30-34    57
25-29    51
20-24    48
15-19    29
10-14    28
40-44    22
35-39    19
0-4       8
50-54     8
5-9       4
45-49     3
Name: count, dtype: int64

Corrected value distribution for 'inv-nodes':
inv-nodes
0-2      209
3-5       34
6-8       17
9-11       7
15-17      6
12-14      3
24-26      1
Name: count, dtype: int64


### Q3

In [10]:
# 加载Excel文件
variables_df = pd.read_excel('variables.xlsx')

ind2val = {}
current_index = 0

# 对每个变量的每个值进行遍历并分配索引
for i, row in variables_df.iterrows():
    variable_name = row['Variable Name']
    values = row['Description'].split(', ')
    for value in values:
        ind2val[current_index] = f"{variable_name}={value}"
        current_index += 1

# 创建从值到索引的逆映射字典
val2ind = {v: k for k, v in ind2val.items()}

# 替换数据集中的文本属性
for column in df.columns:
    if column in variables_df['Variable Name'].values:
        # 为每个列值构建一个映射函数
        df[column] = df[column].apply(lambda x: val2ind[f"{column}={x}"])

# 打印ind2val字典
for key in list(ind2val.keys()):
    print(key, ind2val[key])

0 Class=no-recurrence-events
1 Class=recurrence-events
2 age=10-19
3 age=20-29
4 age=30-39
5 age=40-49
6 age=50-59
7 age=60-69
8 age=70-79
9 age=80-89
10 age=90-99
11 menopause=lt40
12 menopause=ge40
13 menopause=premeno
14 tumor-size=0-4
15 tumor-size=5-9
16 tumor-size=10-14
17 tumor-size=15-19
18 tumor-size=20-24
19 tumor-size=25-29
20 tumor-size=30-34
21 tumor-size=35-39
22 tumor-size=40-44
23 tumor-size=45-49
24 tumor-size=50-54
25 tumor-size=55-59
26 inv-nodes=0-2
27 inv-nodes=3-5
28 inv-nodes=6-8
29 inv-nodes=9-11
30 inv-nodes=12-14
31 inv-nodes=15-17
32 inv-nodes=18-20
33 inv-nodes=21-23
34 inv-nodes=24-26
35 inv-nodes=27-29
36 inv-nodes=30-32
37 inv-nodes=33-35
38 inv-nodes=36-39
39 node-caps=yes
40 node-caps=no
41 deg-malig=1
42 deg-malig=2
43 deg-malig=3
44 breast=left
45 breast=right
46 breast-quad=left_up
47 breast-quad=left_low
48 breast-quad=right_up
49 breast-quad=right_low
50 breast-quad=central
51 irradiat=yes
52 irradiat=no


## 任务二

### Q1

In [11]:
# 设置支持度阈值
min_support = 0.4

# 计算支持度
def calculate_support(df, itemset):
    mask = np.ones(len(df), dtype=bool)
    for item in itemset:
        mask = mask & (df[item[0]] == item[1])
    return mask.sum() / len(df)

# 生成初始单项集
def generate_initial_itemsets(df):
    itemsets = []
    for col in df.columns:
        if col != 'id':
            unique_values = df[col].unique()
            for val in unique_values:
                itemsets.append([(col, val)])
    return itemsets

# 合并项集
def merge_itemsets(itemset1, itemset2):
    return list(set(itemset1).union(set(itemset2)))

# 生成候选项集
def generate_candidates(itemsets, length):
    candidates = []
    for i in range(len(itemsets)):
        for j in range(i+1, len(itemsets)):
            if len(set(itemsets[i]).intersection(set(itemsets[j]))) == length - 2:
                candidate = merge_itemsets(itemsets[i], itemsets[j])
                if len(candidate) == length:
                    candidates.append(candidate)
    return candidates

# 生成所有组合
def generate_combinations(items, length):
    if length == 0:
        return [[]]
    combinations = []
    for i in range(len(items)):
        head = items[i]
        for tail in generate_combinations(items[i + 1:], length - 1):
            combinations.append([head] + tail)
    return combinations

# Apriori算法
def apriori(df, min_support):
    freq_itemsets = []
    itemsets = generate_initial_itemsets(df)
    k = 1

    while itemsets:
        freq_itemsets_k = []
        for itemset in itemsets:
            support = calculate_support(df, itemset)
            if support >= min_support:
                freq_itemsets_k.append(itemset)
        freq_itemsets.extend(freq_itemsets_k)
        itemsets = generate_candidates(freq_itemsets_k, k+1)
        k += 1

    return pd.Series(freq_itemsets).drop_duplicates().tolist()

# 运行Apriori算法
frequent_itemsets = apriori(df, min_support)

# 输出结果
for itemset in frequent_itemsets:
    print(f"频繁项集: {itemset}, 支持度: {calculate_support(df, itemset)}")


频繁项集: [('Class', 0)], 支持度: 0.7075812274368231
频繁项集: [('menopause', 13)], 支持度: 0.5379061371841155
频繁项集: [('menopause', 12)], 支持度: 0.44404332129963897
频繁项集: [('inv-nodes', 26)], 支持度: 0.7545126353790613
频繁项集: [('node-caps', 40)], 支持度: 0.7978339350180506
频繁项集: [('deg-malig', 42)], 支持度: 0.4657039711191336
频繁项集: [('breast', 44)], 支持度: 0.5234657039711191
频繁项集: [('breast', 45)], 支持度: 0.47653429602888087
频繁项集: [('irradiat', 52)], 支持度: 0.776173285198556
频繁项集: [('Class', 0), ('inv-nodes', 26)], 支持度: 0.5992779783393501
频繁项集: [('Class', 0), ('node-caps', 40)], 支持度: 0.6173285198555957
频繁项集: [('Class', 0), ('irradiat', 52)], 支持度: 0.592057761732852
频繁项集: [('inv-nodes', 26), ('menopause', 13)], 支持度: 0.4007220216606498
频繁项集: [('node-caps', 40), ('menopause', 13)], 支持度: 0.4223826714801444
频繁项集: [('irradiat', 52), ('menopause', 13)], 支持度: 0.4007220216606498
频繁项集: [('node-caps', 40), ('inv-nodes', 26)], 支持度: 0.7220216606498195
频繁项集: [('breast', 44), ('inv-nodes', 26)], 支持度: 0.4007220216606498
频繁项集: [('inv-

### Q2

In [12]:
# 最小置信度阈值
min_confidence = 0.75

# 计算支持度
def calculate_support(df, itemset):
    mask = np.ones(len(df), dtype=bool)
    for item in itemset:
        mask = mask & (df[item[0]] == item[1])
    return mask.sum() / len(df)

# 计算置信度
def calculate_confidence(df, antecedent, consequent):
    support_antecedent = calculate_support(df, antecedent)
    support_antecedent_consequent = calculate_support(df, antecedent + consequent)
    return support_antecedent_consequent / support_antecedent

# 计算提升度
def calculate_lift(df, antecedent, consequent):
    support_antecedent_consequent = calculate_support(df, antecedent + consequent)
    support_antecedent = calculate_support(df, antecedent)
    support_consequent = calculate_support(df, consequent)
    return support_antecedent_consequent / (support_antecedent * support_consequent)

# 提取强关联规则
def extract_strong_rules(df, frequent_itemsets, min_confidence):
    strong_rules = []
    for itemset in frequent_itemsets:
        if ('Class', 0) not in itemset:
            continue
        antecedents = [x for x in itemset if x != ('Class', 0)]
        if antecedents:
            confidence = calculate_confidence(df, antecedents, [('Class', 0)])
            if confidence >= min_confidence:
                lift = calculate_lift(df, antecedents, [('Class', 0)])
                strong_rules.append((antecedents, ('Class', 0), confidence, lift))
    return pd.Series(strong_rules).drop_duplicates().tolist()

strong_rules = extract_strong_rules(df, frequent_itemsets, min_confidence)

# 输出结果
for antecedents, consequent, confidence, lift in strong_rules:
    print(f"规则: {antecedents} -> {consequent}, 置信度: {confidence}, 提升度: {lift}")


规则: [('inv-nodes', 26)] -> ('Class', 0), 置信度: 0.7942583732057417, 提升度: 1.1224978029489308
规则: [('node-caps', 40)] -> ('Class', 0), 置信度: 0.7737556561085973, 提升度: 1.0935220241942933
规则: [('irradiat', 52)] -> ('Class', 0), 置信度: 0.7627906976744185, 提升度: 1.0780256288561936
规则: [('node-caps', 40), ('inv-nodes', 26)] -> ('Class', 0), 置信度: 0.7999999999999999, 提升度: 1.1306122448979592
规则: [('inv-nodes', 26), ('irradiat', 52)] -> ('Class', 0), 置信度: 0.8166666666666667, 提升度: 1.1541666666666668
规则: [('node-caps', 40), ('irradiat', 52)] -> ('Class', 0), 置信度: 0.8074866310160427, 提升度: 1.1411928407726726
规则: [('inv-nodes', 26), ('node-caps', 40)] -> ('Class', 0), 置信度: 0.7999999999999999, 提升度: 1.1306122448979592
规则: [('node-caps', 40), ('irradiat', 52), ('inv-nodes', 26)] -> ('Class', 0), 置信度: 0.8238636363636364, 提升度: 1.1643378942486085


### Q3

从频繁项集来看，无结节冒，未采取放疗，受侵淋巴结数目在0-2，疾病没有复发这些特征为数据集中较为常见的。

将提取出的关联规则替换为原本的文字得到如下结果：
1. inv-nodes=0-2 -> Class=no-recurrence-events
2. node-caps=no -> Class=no-recurrence-events
3. irradiat=no -> Class=no-recurrence-events
4. node-caps=no, inv-nodes=0-2 -> Class=no-recurrence-events
5. inv-nodes=0-2, irradiat=no -> Class=no-recurrence-events
6. node-caps=no, irradiat=no -> Class=no-recurrence-events
7. inv-nodes=0-2, node-caps=no -> Class=no-recurrence-events
8. node-caps=no, irradiat=no, inv-nodes=0-2 -> Class=no-recurrence-events

由上面的结果可以得到：无结节冒，未采取放疗，受侵淋巴结数目在0-2这些特征存在的情况下，不复发的概率较高。